# CMPE 255-03, Fall 2024
## Steam CS2 and TF2 Project

**Due Date:** 6:00pm on Oct 30th, 2024

EDA Project for Steam Items in CS2 and TF2

---

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob

Please download fashion_mnist1.csv.
The dataset has 10K rows and 785 columns.

In [9]:
# Load the dataset
data = pd.read_csv('reviews_440.csv')
data.head()

# data.info()

,recommendationid,gameid,steamid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,language,...,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location,primarily_steam_deck
0,176534183,440,76561199075358962,0,6,26921,27,26921.0,1728195517,english,...,0,0,0.00000,0,False,False,False,True,NaN,False
1,176533586,440,76561198127158652,1124,68,9220,0,9220.0,1726398332,czech,...,1,0,0.52381,0,False,False,False,True,NaN,False
2,176533564,440,76561199178829199,8,2,36,0,36.0,1645151002,portuguese,...,0,0,0.00000,0,False,False,False,True,NaN,False
3,176533493,440,76561199783055220,0,1,677,516,677.0,1728261916,english,...,0,0,0.00000,0,False,False,False,True,NaN,False
4,176533284,440,76561198157582622,165,9,162721,4986,162721.0,1728259830,english,...,0,0,0.00000,0,False,False,False,True,NaN,False


#### 1. Remove Duplicates from CSV Files

In [10]:
# List of file paths
# C:\Users\chaoa\Downloads\CMPE255\CS2Reviews\reviews_730_01.csv
file_paths = sorted(glob.glob('./CS2Reviews/*.csv'))  # Sort if order matters

print(file_paths)

# Start by loading the first file as the initial base
combined_df = pd.read_csv(file_paths[0])

# Iterate over remaining files and combine without duplicates
for file_path in file_paths[1:]:
    # Load the next file
    next_df = pd.read_csv(file_path)
    
    # Remove entries in `next_df` that already exist in `combined_df`
    next_df = next_df[~next_df['recommendationid'].isin(combined_df['recommendationid'])]
    
    # Concatenate the unique entries to `combined_df`
    combined_df = pd.concat([combined_df, next_df], ignore_index=True)

# Save the final deduplicated result
combined_df.to_csv("./CS2Reviews/CombinedCS2Reviews.csv", index=False)

['./CS2Reviews\\CombinedCS2Reviews.csv', './CS2Reviews\\reviews_730_01.csv', './CS2Reviews\\reviews_730_02.csv', './CS2Reviews\\reviews_730_03.csv', './CS2Reviews\\reviews_730_04.csv', './CS2Reviews\\reviews_730_05.csv', './CS2Reviews\\reviews_730_06.csv', './CS2Reviews\\reviews_730_07.csv', './CS2Reviews\\reviews_730_08.csv', './CS2Reviews\\reviews_730_09.csv', './CS2Reviews\\reviews_730_10.csv', './CS2Reviews\\reviews_730_11.csv', './CS2Reviews\\reviews_730_12.csv', './CS2Reviews\\reviews_730_13.csv', './CS2Reviews\\reviews_730_14.csv', './CS2Reviews\\reviews_730_15.csv', './CS2Reviews\\reviews_730_16.csv', './CS2Reviews\\reviews_730_17.csv', './CS2Reviews\\reviews_730_18.csv', './CS2Reviews\\reviews_730_19.csv', './CS2Reviews\\reviews_730_20.csv', './CS2Reviews\\reviews_730_21.csv', './CS2Reviews\\reviews_730_22.csv', './CS2Reviews\\reviews_730_23.csv', './CS2Reviews\\reviews_730_combined.csv']


C:\Users\chaoa\AppData\Local\Temp\ipykernel_31988\1463797543.py:10: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv(file_paths[0])
C:\Users\chaoa\AppData\Local\Temp\ipykernel_31988\1463797543.py:15: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  next_df = pd.read_csv(file_path)
C:\Users\chaoa\AppData\Local\Temp\ipykernel_31988\1463797543.py:15: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  next_df = pd.read_csv(file_path)
C:\Users\chaoa\AppData\Local\Temp\ipykernel_31988\1463797543.py:15: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  next_df = pd.read_csv(file_path)
C:\Users\chaoa\AppData\Local\Temp\ipykernel_31988\1463797543.py:15: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  next_

KeyboardInterrupt: 

### Checking files

In [6]:
# 1. Load the dataset
df = pd.read_csv('./CS2Reviews/CombinedCS2Reviews.csv')

df.info()

cv = pd.read_csv('./CS2Reviews/reviews_730_combined.csv')

cv.info()

C:\Users\chaoa\AppData\Local\Temp\ipykernel_31988\776911025.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./CS2Reviews/CombinedCS2Reviews.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8335510 entries, 0 to 8335509
Data columns (total 24 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   recommendationid             int64  
 1   gameid                       int64  
 2   steamid                      int64  
 3   num_games_owned              int64  
 4   num_reviews                  int64  
 5   playtime_forever             float64
 6   playtime_last_two_weeks      float64
 7   playtime_at_review           float64
 8   last_played                  float64
 9   language                     object 
 10  review_length                int64  
 11  timestamp_created            int64  
 12  timestamp_updated            int64  
 13  voted_up                     bool   
 14  votes_up                     int64  
 15  votes_funny                  int64  
 16  weighted_vote_score          float64
 17  comment_count                int64  
 18  steam_purchase               bool   
 19  

C:\Users\chaoa\AppData\Local\Temp\ipykernel_31988\776911025.py:6: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  cv = pd.read_csv('./CS2Reviews/reviews_730_combined.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8367210 entries, 0 to 8367209
Data columns (total 24 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   recommendationid             int64  
 1   gameid                       int64  
 2   steamid                      int64  
 3   num_games_owned              int64  
 4   num_reviews                  int64  
 5   playtime_forever             float64
 6   playtime_last_two_weeks      float64
 7   playtime_at_review           float64
 8   last_played                  float64
 9   language                     object 
 10  review_length                int64  
 11  timestamp_created            int64  
 12  timestamp_updated            int64  
 13  voted_up                     bool   
 14  votes_up                     int64  
 15  votes_funny                  int64  
 16  weighted_vote_score          float64
 17  comment_count                int64  
 18  steam_purchase               bool   
 19  

#### Removing Duplicate Blobs at the beginning or end of files (situations where requests halted prematurely and overlap occurred)

In [2]:
file_paths = sorted(glob.glob('./CS2Prices/*.csv'))  # Sort if order matters

print(file_paths)

# Start by loading the first file as the initial base
combined_df = pd.read_csv(file_paths[0])

# Iterate over remaining files and combine without duplicates
for file_path in file_paths[1:]:
    # Load the next file
    next_df = pd.read_csv(file_path)
    
    # Remove entries in `next_df` that already exist in `combined_df`
    next_df = next_df[~next_df.set_index(['Item Name', 'Date']).index.isin(combined_df.set_index(['Item Name', 'Date']).index)]
    
    # Concatenate the unique entries to `combined_df`
    combined_df = pd.concat([combined_df, next_df], ignore_index=True)

# Save the final deduplicated result
combined_df.to_csv("./CS2Prices/CombinedCS2Prices.csv", index=False)

NameError: name 'glob' is not defined

In [13]:
df = pd.read_csv('./CS2Prices/CombinedCS2Prices.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15524405 entries, 0 to 15524404
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Item Name  object 
 1   Parent     object 
 2   Date       object 
 3   Price      float64
 4   Volume     int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 592.2+ MB


TF2 Prices

In [8]:
file_paths = ["./TF2Prices/steam_440_price_history_1_31.csv", "./TF2Prices/steam_price_history_tf2_32-103.csv"]

# Start by loading the first file as the initial base
combined_df = pd.read_csv(file_paths[0])

# Iterate over remaining files and combine without duplicates
for file_path in file_paths[1:]:
    # Load the next file
    next_df = pd.read_csv(file_path)
    
    # Remove entries in `next_df` that already exist in `combined_df`
    next_df = next_df[~next_df.set_index(['Item Name', 'Parent', 'Date']).index.isin(combined_df.set_index(['Item Name', 'Parent', 'Date']).index)]
    
    # Concatenate the unique entries to `combined_df`
    combined_df = pd.concat([combined_df, next_df], ignore_index=True)

# Save the final deduplicated result
combined_df.to_csv("./TF2Prices/CombinedTF2Prices.csv", index=False)

In [11]:
# Accounting for items with multiple parents

combined_df = pd.read_csv("./TF2Prices/CombinedTF2Prices.csv")

# # Identify rows with duplicates
# duplicate_groups = combined_df.groupby(['Item Name', 'Date']).filter(lambda x: len(x) > 1)

# print("Combined rows (only those with duplicates):")
# print(duplicate_groups)

# Group by 'Item Name' and 'Date' and aggregate with Parent as a list
combined_df = combined_df.groupby(['Item Name', 'Date'], as_index=False).agg({
    'Parent': lambda x: list(x.unique()),
    'Price': 'first',
    'Volume': 'first'
})

# Filter rows where the 'Parent' list has more than one entry
multiple_parents = combined_df[combined_df['Parent'].apply(len) > 1]

print("Rows with multiple parents:")
print(multiple_parents)

Rows with multiple parents:
                        Item Name                Date  \
0        A Color Similar to Slate  Apr 01 2013 01: +0   
1        A Color Similar to Slate  Apr 01 2014 01: +0   
2        A Color Similar to Slate  Apr 01 2015 01: +0   
3        A Color Similar to Slate  Apr 01 2016 01: +0   
4        A Color Similar to Slate  Apr 01 2017 01: +0   
...                           ...                 ...   
1071332         Zepheniah's Greed  Sep 30 2024 18: +0   
1071333         Zepheniah's Greed  Sep 30 2024 19: +0   
1071334         Zepheniah's Greed  Sep 30 2024 20: +0   
1071335         Zepheniah's Greed  Sep 30 2024 21: +0   
1071336         Zepheniah's Greed  Sep 30 2024 23: +0   

                                                    Parent  Price  Volume  
0        [Mann Co. Supply Crate Series #9, Mann Co. Sup...  0.931      18  
1        [Mann Co. Supply Crate Series #9, Mann Co. Sup...  0.854      12  
2        [Mann Co. Supply Crate Series #9, Mann Co. Sup... 

In [12]:
# Export Combined Datasets
combined_df.to_json('./TF2Prices/CombinedTF2Prices_MultiParents.json', orient='records', lines=True)

#### 2. Feature Engineering

In [74]:
cs = pd.read_csv('./CS2Prices/CombinedCS2Prices.csv')

tf = pd.read_json('./TF2Prices/CombinedTF2Prices_MultiParents.json', orient='records', lines=True)

##### 2.1 DateTime --> Day / Month / Year Columns (Chronological Data)

In [64]:
from datetime import datetime, timedelta

def convert_date_time(date_string):
    # Parse the date and time, ignoring the timezone offset
    dt = datetime.strptime(date_string[:-3], "%b %d %Y %H:")

    # Add the timezone offset
    offset = int(date_string[-2:])
    dt = dt + timedelta(hours=offset)

    return dt

date_string = "Oct 27 2024 09: +0"
converted_dt = convert_date_time(date_string)
print(converted_dt)

2024-10-27 09:00:00


CS2

In [37]:
cs['Iso_Date'] = cs['Date'].apply(convert_date_time) # ISO 8601 format (YYYY-MM-DD)

cs['Day'] = cs['Iso_Date'].dt.day
cs['Month'] = cs['Iso_Date'].dt.month # integer month
cs['Year'] = cs['Iso_Date'].dt.year

print(cs[['Date', 'Day', 'Month', 'Year', 'Iso_Date']])

                        Date  Day  Month  Year            Iso_Date
0         Aug 14 2013 01: +0   14      8  2013 2013-08-14 01:00:00
1         Aug 15 2013 01: +0   15      8  2013 2013-08-15 01:00:00
2         Aug 16 2013 01: +0   16      8  2013 2013-08-16 01:00:00
3         Aug 17 2013 01: +0   17      8  2013 2013-08-17 01:00:00
4         Aug 18 2013 01: +0   18      8  2013 2013-08-18 01:00:00
...                      ...  ...    ...   ...                 ...
15524400  Oct 25 2024 13: +0   25     10  2024 2024-10-25 13:00:00
15524401  Oct 25 2024 17: +0   25     10  2024 2024-10-25 17:00:00
15524402  Oct 26 2024 01: +0   26     10  2024 2024-10-26 01:00:00
15524403  Oct 26 2024 05: +0   26     10  2024 2024-10-26 05:00:00
15524404  Oct 26 2024 11: +0   26     10  2024 2024-10-26 11:00:00

[15524405 rows x 5 columns]


In [39]:
cs.to_csv("./CS2Prices/CombinedCS2Prices_Dates.csv", index=False)

TF2

In [73]:
tf['Iso_Date'] = tf['Date'].apply(convert_date_time) # ISO 8601 format (YYYY-MM-DD)

tf['Day'] = tf['Iso_Date'].dt.day
tf['Month'] = tf['Iso_Date'].dt.month # integer month
tf['Year'] = tf['Iso_Date'].dt.year

print(tf[['Date', 'Day', 'Month', 'Year', 'Iso_Date']])

                       Date  Day  Month  Year            Iso_Date
0        Apr 01 2013 01: +0    1      4  2013 2013-04-01 01:00:00
1        Apr 01 2014 01: +0    1      4  2014 2014-04-01 01:00:00
2        Apr 01 2015 01: +0    1      4  2015 2015-04-01 01:00:00
3        Apr 01 2016 01: +0    1      4  2016 2016-04-01 01:00:00
4        Apr 01 2017 01: +0    1      4  2017 2017-04-01 01:00:00
...                     ...  ...    ...   ...                 ...
1071332  Sep 30 2024 18: +0   30      9  2024 2024-09-30 18:00:00
1071333  Sep 30 2024 19: +0   30      9  2024 2024-09-30 19:00:00
1071334  Sep 30 2024 20: +0   30      9  2024 2024-09-30 20:00:00
1071335  Sep 30 2024 21: +0   30      9  2024 2024-09-30 21:00:00
1071336  Sep 30 2024 23: +0   30      9  2024 2024-09-30 23:00:00

[1071337 rows x 5 columns]


In [71]:
tf['Iso_Date'] = tf['Iso_Date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
tf.to_json('./TF2Prices/CombinedTF2Prices_MultiParents_Dates.json', orient='records', lines=True)

##### 2.2 Lifespan = Current date - first date (Chronological Data)

CS2

In [54]:
cs = pd.read_csv('./CS2Prices/CombinedCS2Prices_Dates.csv')

# cs.info()
# Convert 'Iso_Date' to datetime
cs['Iso_Date'] = pd.to_datetime(cs['Iso_Date'])

# Find the first occurrence of each item
first_occurrence = cs.groupby('Item Name')['Iso_Date'].transform('min')

# Calculate Lifespan by subtracting the first occurrence date from the current row's date
cs['Lifespan'] = cs['Iso_Date'] - first_occurrence

# Optional: Convert Lifespan to total days
cs['Lifespan_Days'] = cs['Lifespan'].dt.days

# Display the DataFrame
# print(cs)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15524405 entries, 0 to 15524404
Data columns (total 9 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Item Name  object 
 1   Parent     object 
 2   Date       object 
 3   Price      float64
 4   Volume     int64  
 5   Iso_Date   object 
 6   Day        int64  
 7   Month      int64  
 8   Year       int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 1.0+ GB
                                          Item Name        Parent  \
0                                 CS:GO Weapon Case           NaN   
1                                 CS:GO Weapon Case           NaN   
2                                 CS:GO Weapon Case           NaN   
3                                 CS:GO Weapon Case           NaN   
4                                 CS:GO Weapon Case           NaN   
...                                             ...           ...   
15524400  StatTrak M4A1-S | Vaporwave (Factory New)  Gallery Case   
15524401

In [55]:
cs.to_csv("./CS2Prices/CombinedCS2Prices_Dates_Lifetime.csv", index=False)

TF2

In [72]:
tf = pd.read_json('./TF2Prices/CombinedTF2Prices_MultiParents_Dates.json', orient='records', lines=True)

print(tf[:5])

# tf['Iso_Date'] = pd.to_datetime(tf['Iso_Date'])

# print(tf['Iso_Date'][:5])

# # Find the first occurrence of each item
# first_occurrence = tf.groupby('Item Name')['Iso_Date'].transform('min')

# # Calculate Lifespan by subtracting the first occurrence date from the current row's date
# tf['Lifespan'] = tf['Iso_Date'] - first_occurrence

# # print(tf['Lifespan'])

# # Optional: Convert Lifespan to total days
# tf['Lifespan_Days'] = tf['Lifespan'].dt.days

# print(tf)

                  Item Name                Date  \
0  A Color Similar to Slate  Apr 01 2013 01: +0   
1  A Color Similar to Slate  Apr 01 2014 01: +0   
2  A Color Similar to Slate  Apr 01 2015 01: +0   
3  A Color Similar to Slate  Apr 01 2016 01: +0   
4  A Color Similar to Slate  Apr 01 2017 01: +0   

                                              Parent  Price  Volume  \
0  [Mann Co. Supply Crate Series #9, Mann Co. Sup...  0.931      18   
1  [Mann Co. Supply Crate Series #9, Mann Co. Sup...  0.854      12   
2  [Mann Co. Supply Crate Series #9, Mann Co. Sup...  0.835      19   
3  [Mann Co. Supply Crate Series #9, Mann Co. Sup...  1.250       5   
4  [Mann Co. Supply Crate Series #9, Mann Co. Sup...  1.009       6   

        Iso_Date  Day  Month  Year  
0  1364778000000    1      4  2013  
1  1396314000000    1      4  2014  
2  1427850000000    1      4  2015  
3  1459472400000    1      4  2016  
4  1491008400000    1      4  2017  


In [57]:
tf.to_json('./TF2Prices/CombinedTF2Prices_MultiParents_Dates_Lifetime.json', orient='records', lines=True)

##### 2.3 Concurrent Players (Social Data)

CS2

In [ ]:
cs_players = pd.read_csv('./PlayerCount/cs2playercount.csv')

TF2

In [ ]:
tf_players = pd.read_csv('./PlayerCount/tf2playercount.csv')

##### 2.4 Game Price (Financial Data)

CS2

In [ ]:
cs_price = pd.read_csv('./GamePrice/cs2gamepricehistory.csv')

TF2

Use Orange Box (TF2 included in bundle)
F2P June 23, 2011

In [ ]:
tf_price = pd.read_csv('./GamePrice/orangeboxgamepricehistory.csv')

##### Review Sentiment

CS2

TF2

#### 3. Outlier Detection and Handling

#### 4. Pearson Coefficients and Heatmap

#### 5. Machine Learning Algorithms

##### 5.1 Linear Regression